# Upload sequencing data to SRA
This Python Jupyter notebook uploads the sequencing data to the NIH [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra), or SRA.

## Create BioProject and BioSamples
The first step was done manually to create the BioProject and BioSamples.
Note that for new future uploads related to the RBD DMS, you may be able to use the existing BioProject, but since this is the first entries in these project I needed to create a new BioProject.

To create these, I went to the [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra) and signed in using the box at the upper right of the webpage, and then went to the [SRA Submission Portal](https://submit.ncbi.nlm.nih.gov/subs/sra/).
I then manually completed the first five steps, which define the project and samples.

## Create submission sheet
The sixth step is to create the submission sheet in `*.tsv` format, which is done by the following code.

First, import Python modules:

In [1]:
import ftplib
import os
import tarfile
import datetime

import natsort

import pandas as pd

import yaml

Read the configuration for the analysis:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

Read the PacBio runs:

In [3]:
pacbio_runs_file = os.path.join('../', config['pacbio_runs'])

print(f"Reading PacBio runs from {pacbio_runs_file}")

pacbio_runs = (
    pd.read_csv(pacbio_runs_file)
    .assign(ccs_file=lambda x: f"../{config['ccs_dir']}/" + x['library'] + '_' + x['run'] + '_ccs.fastq.gz')
    )

pacbio_runs.head()

Reading PacBio runs from ../data/PacBio_runs.csv


,library,run,subreads,ccs_file
0,lib1,200916_1,/fh/fast/bloom_j/SR/ngs/pacbio/200924_TylerSta...,../results/ccs/lib1_200916_1_ccs.fastq.gz
1,lib2,200916_1,/fh/fast/bloom_j/SR/ngs/pacbio/200924_TylerSta...,../results/ccs/lib2_200916_1_ccs.fastq.gz
2,lib1,200916_2,/fh/fast/bloom_j/SR/ngs/pacbio/200929_TylerSta...,../results/ccs/lib1_200916_2_ccs.fastq.gz
3,lib2,200916_2,/fh/fast/bloom_j/SR/ngs/pacbio/200929_TylerSta...,../results/ccs/lib2_200916_2_ccs.fastq.gz
4,lib1,201110,/fh/fast/bloom_j/computational_notebooks/tstar...,../results/ccs/lib1_201110_ccs.fastq.gz


Next make submission entries for the PacBio CCSs:

In [18]:
pacbio_submissions = (
    pacbio_runs
    .assign(
        biosample_accession='SAMN18316101',
        library_ID=lambda x: x['library'] + '_sarbecovirus_RBD_barcode_association',  # unique library ID
        title='PacBio sequencing for barcode-RBD association within a library of yeast-displayed sarbecovirus RBDs',
        library_strategy='Synthetic-Long-Read',
        library_source='SYNTHETIC',
        library_selection='Restriction Digest',
        library_layout='single',
        platform='PACBIO_SMRT',
        instrument_model='PacBio Sequel',
        design_description='Restriction digest of plasmids carrying barcoded RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['ccs_file'],      
        )
    .drop(columns=pacbio_runs.columns)
    )

pacbio_submissions.head()

,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath
0,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_200916_1_ccs.fastq.gz
1,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib2_200916_1_ccs.fastq.gz
2,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_200916_2_ccs.fastq.gz
3,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib2_200916_2_ccs.fastq.gz
4,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_201110_ccs.fastq.gz


Now concatenate the PacBio submissions into tidy format (one line per file), make sure all the files exist, and also make short name versions of them that lack the path:

In [19]:
submissions_tidy = (
    pd.concat([pacbio_submissions], ignore_index=True)
    .assign(file_exists=lambda x: x['filename_fullpath'].map(os.path.isfile),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename),
            )
    )

assert submissions_tidy['file_exists'].all(), submissions_tidy.query('file_exists == False')

submissions_tidy.head()

,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,file_exists,filename
0,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_200916_1_ccs.fastq.gz,True,lib1_200916_1_ccs.fastq.gz
1,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib2_200916_1_ccs.fastq.gz,True,lib2_200916_1_ccs.fastq.gz
2,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_200916_2_ccs.fastq.gz,True,lib1_200916_2_ccs.fastq.gz
3,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib2_200916_2_ccs.fastq.gz,True,lib2_200916_2_ccs.fastq.gz
4,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,../results/ccs/lib1_201110_ccs.fastq.gz,True,lib1_201110_ccs.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [21]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,3
1,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,3


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [22]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'file_exists', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

submissions_wide

,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3
0,SAMN18316101,lib1_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,lib1_200916_1_ccs.fastq.gz,lib1_200916_2_ccs.fastq.gz,lib1_201110_ccs.fastq.gz
1,SAMN18316101,lib2_sarbecovirus_RBD_barcode_association,PacBio sequencing for barcode-RBD association ...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel,Restriction digest of plasmids carrying barcod...,fastq,lib2_200916_1_ccs.fastq.gz,lib2_200916_2_ccs.fastq.gz,lib2_201110_ccs.fastq.gz


Now write the wide submissions data frame to a `*.tsv` file:

In [23]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

This submission sheet was then manually uploaded in Step 6 of the SRA submission wizard (*SRA metadata*).

## Upload the actual files
Step 7 of the SRA submission wizard is to upload the files.
In order to do this, we first make a `*.tar` file with all of the files.
Since this takes a long time, we only create the file if it doesn't already exist, so it is only created the first time this notebook is run.
**Note that this will cause a problem if you add more sequencing files to upload after running the notebook, in that case the cell below will need to altered.**

In [9]:
tar_filename = 'SRA_submission.tar'

if os.path.isfile(tar_filename):
    print(f"{tar_filename} already exists, not creating it again")
else:
    try:
        with tarfile.open(tar_filename, mode='w') as f:
            for i, tup in enumerate(submissions_tidy.itertuples()):
                print(f"Adding file {i + 1} of {len(submissions_tidy)} to {tar_filename}")
                f.add(tup.filename_fullpath, arcname=tup.filename)
            print(f"Added all files to {tar_filename}")
    except:
        if os.path.isfile(tar_filename):
            os.remove(tar_filename)
        raise

Adding file 1 of 6 to SRA_submission.tar
Adding file 2 of 6 to SRA_submission.tar
Adding file 3 of 6 to SRA_submission.tar
Adding file 4 of 6 to SRA_submission.tar
Adding file 5 of 6 to SRA_submission.tar
Adding file 6 of 6 to SRA_submission.tar
Added all files to SRA_submission.tar


See the size of the `*.tar` file to upload and make sure it has the expected files:

In [10]:
print(f"The size of {tar_filename} is {os.path.getsize(tar_filename) / 1e9:.1f} GB")

with tarfile.open(tar_filename) as f:
    files_in_tar = set(f.getnames())
if files_in_tar == set(submissions_tidy['filename']):
    print(f"{tar_filename} contains all {len(files_in_tar)} expected files.")
else:
    raise ValueError(f"{tar_filename} does not have all the expected files.")

The size of SRA_submission.tar is 0.3 GB
SRA_submission.tar contains all 6 expected files.


The SRA instructions then give several ways to upload; we will do it using the FTP method.
First, specify the FTP address, username, password, and subfolder given by the SRA submission wizard instructions.
In order to avoid having the password be public here, that is in a separate text file that is **not** included in the GitHub repo (so this needs to be run in Jesse's directory that has this password):

In [16]:
# the following are provided by SRA wizard insturctions
ftp_address = 'ftp-private.ncbi.nlm.nih.gov'
ftp_username = 'subftp'
ftp_account_folder = 'uploads/tyler.n.starr_gmail.com_JS1IV8bi'
with open('ftp_password.txt') as f:
    ftp_password = f.read().strip()
    
# meaningful name for subfolder
ftp_subfolder = 'sarbecovirus_RBD_barcode_association'

Now create FTP connection and upload the TAR file.
Note that this takes a while.
If you are worried that it will timeout given the size of your file, you can run this notebook via `slurm` so there is no timing out:

In [17]:
print(f"Starting upload at {datetime.datetime.now()}")

with ftplib.FTP(ftp_address) as ftp:
    ftp.login(user=ftp_username,
              passwd=ftp_password,
              )
    ftp.cwd(ftp_account_folder)
    ftp.mkd(ftp_subfolder)
    ftp.cwd(ftp_subfolder)
    with open(tar_filename, 'rb') as f:
        ftp.storbinary(f"STOR {tar_filename}", f)
        
print(f"Finished upload at {datetime.datetime.now()}")

Starting upload at 2021-03-15 16:33:35.942827
Finished upload at 2021-03-15 16:34:46.371174


Finally, used the SRA wizard to select the `*.tar` archive and complete the submission.
Note that there is a warning of missing files since everything was uploaded as a `*.tar` rather than individual files.
They should all be found when you hit the button to proceed and the `*.tar` is unpacked.

There was then a message that the submission was processing, and data would be released immediately upon processing.
The submission number is `SUB7594564`.